In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.colors as cc
from matplotlib.colors import ListedColormap

# creates the four colors for our simulation, EMPTY is white,
# TREE is green, FIRE is red and BURNT is black
colors = cc.ColorConverter.colors
cols = [colors['w'],
        colors['g'],
        colors['r'],
        colors['k']]
ff1 = ListedColormap(cols)

# TODO revise this method to include a boolean parameter called torus
#  when true, the function should wrap around the edges, when false
#  return the function as written.
def von_neuman_neighbors(i, j, size, torus):
    n = []
    if torus:
        if i == 0:
            n.append((size - 1, j))
        if j == 0:
            n.append((i, size - 1))
        if i == size - 1:
            n.append((0, j))
        if j == size - 1:
            n.append((i, 0))
    if i > 0:
        n.append((i - 1, j))
    if j > 0:
        n.append((i, j - 1))
    if i < size - 1:
        n.append((i + 1, j))
    if j < size - 1:
        n.append((i, j + 1))
    return n

# TODO write this method to include all eight neighbors. Use the 
#  torus parameter as described above
def moore_neighbors(i, j, size, torus):
    n = []
    if torus:
        if i == 0:
            n.append((size - 1, j))
        if j == 0:
            n.append((i, size - 1))
        if i == size - 1:
            n.append((0, j))
        if j == size - 1:
            n.append((i, 0))
            
        if i == 0 and j == 0:
            n.append((size - 1, size - 1))    
        if i == 0 and j > 0:
            n.append((size - 1, j - 1))
        if i == 0 and j < size - 1:
            n.append((size - 1, j + 1))
        if j == 0 and i > 0:
            n.append((i - 1, size - 1))
        if j == 0 and i < size - 1:
            n.append((i + 1, size - 1))
        if i == size - 1 and j == size - 1:
            n.append((0, 0))
        if i == size - 1 and j > 0:
            n.append((0, j - 1))
        if i == size - 1 and j < size - 1:
            n.append((0, j + 1))
        if j == size - 1 and i > 0:
            n.append((i - 1, 0))
        if j == size - 1 and i < 0: 
            n.append((i + 1, 0))
            
        if j == 0 and i == size - 1:
            n.append((0, size - 1))
        if i == 0 and j == size - 1:
            n.append((size - 1, 0))
               
    if i > 0:
        n.append((i - 1, j))
    if j > 0:
        n.append((i, j - 1))
    if i < size - 1:
        n.append((i + 1, j))
    if j < size - 1:
        n.append((i, j + 1))
        
    if i > 0 and j > 0:
        n.append((i - 1, j - 1))
    if i < size - 1 and j < size - 1:
        n.append((i + 1, j + 1))
    if i > 0 and j < size - 1:
        n.append((i - 1, j + 1))
    if j > 0 and i < size - 1:
        n.append((i + 1, j - 1))
    
    return n

class Forest():
    
    states = {"EMPTY":0, "TREE":1, "FIRE":2, "BURNT":3}
    
    def __init__(self, size, torus):
        self.size = size
        self.x = np.arange(size)
        self.y = np.arange(size)
        self.cells = np.zeros((size, size), dtype="int")
        self.time = 0
        self.burnt = 0
        self.initTree = 0
        self.initEmpty = 0
        self.torus = torus
        
        self.burntList = []
        self.treeList = []
        self.emptyList = []
        self.fireList = []
        
    def set_fire(self, locs):
        for p in locs:
            self.cells[p[0] + self.size // 2, p[1] + self.size // 2] = Forest.states["FIRE"]
        
    def random_setup(self, d):
        for i in range(self.size):
            for j in range(self.size):
                if (np.random.random() < d):
                    self.cells[i, j] = Forest.states["TREE"]
                else:
                    self.cells[i, j] = Forest.states["EMPTY"]
        self.initTree = np.count_nonzero(self.cells == 1)
        self.initEmpty = np.count_nonzero(self.cells == 0)
        
    def image_setup(self):
        plt.title("Forest Fire Simulation")
        self.plt = plt.imshow(self.cells, interpolation='nearest', 
                            origin='bottom', 
                            vmin=Forest.states["EMPTY"],
                            vmax=Forest.states["BURNT"], 
                            cmap=ff1)

    def update(self):
        self.time += 1
        
        newcells = np.zeros((self.size, self.size), dtype="int")
        
        
        for i in range(self.size):
            for j in range(self.size):
                neighbors = von_neuman_neighbors(i, j, self.size, self.torus)
                #neighbors = moore_neighbors(i, j, self.size, True)
                # TODO Add in code to make the fire spread. If a tree is near 
                #  a fire cell, it catches. All trees on fire burn out completely in
                #  one timestep.
                if self.cells[i,j] == Forest.states["FIRE"]:
                    newcells[i,j] = Forest.states["BURNT"]
                elif self.cells[i,j] == Forest.states["TREE"]:
                    newcells[i,j] = self.cells[i,j]
                    for n in neighbors:
                        if self.cells[n[0], n[1]] == Forest.states["FIRE"]:
                            newcells[i, j] = Forest.states["FIRE"]
                elif self.cells[i,j] == Forest.states["BURNT"]:
                    rand = np.random.random()
                    if rand >= .999:
                        newcells[i, j] = Forest.states["EMPTY"]
                    else:
                        newcells[i,j] = self.cells[i,j]
                elif self.cells[i,j] == Forest.states["EMPTY"]:
                    rand = np.random.random()
                    if rand >= .999:
                        #newcells[i, j] = Forest.states["TREE"]
                        for n in neighbors:
                            if self.cells[n[0], n[1]] == Forest.states["TREE"]:
                                newcells[i, j] = Forest.states["TREE"]
                    else:
                        newcells[i,j] = self.cells[i,j]
                    
                # TODO Revise to have burnt trees turn into empty cells randomly
                ##Random numbers for each of these. 
                ##For empty cells adjacent. look to the Tree to Fire section
                # TODO Revise to have empty cells grow new trees randomly if they
                #  they are adjacent to a living tree
                    
                    
        # TODO Add in a probability of a lightning strike each round. If it hits a 
        #  tree, the tree is set on fire.
        probLightning = .1
        randL = np.random.random()
        #2 Random numbers between 0 and size - 1, this will be i, j
        l = np.random.randint(0, size - 1)
        m = np.random.randint(0, size - 1)
        if randL < probLightning:
            if newcells[l, m] == Forest.states["TREE"]:
                newcells[l, m] = Forest.states["FIRE"]
        #Check if the cell at i, j is a tree. If it is, set it to Fire
        self.burnt = np.count_nonzero(newcells == 3)   
        self.burntList.append(int(np.count_nonzero(newcells == 3)))
        self.treeList.append(int(np.count_nonzero(newcells == 1)))
        self.emptyList.append(int(np.count_nonzero(newcells == 0)))
        self.fireList.append(int(np.count_nonzero(newcells == 2)))
        self.cells = newcells
        
    def plot(self):
        self.plt.set_data(self.cells)
        return self.plt
        
size = 60
density = 0.55

fig, ax = plt.subplots()
ax.set_ylim(-1, size)
ax.set_xlim(-1, size)

middle = ((0, 0), )

ff = Forest(size, False)
ff.random_setup(density)
ff.set_fire(middle)

################
# ANIMATION
ff.image_setup()

def update(data):
    ff.update()
    return ff.plot(),

def data_gen():
    while True: yield 1

ani = animation.FuncAnimation(fig, update, data_gen, blit=False, interval=200)
plt.show()